In [8]:
import pandas as pd
import jsonlines
import json
import os 

import openai

In [9]:
config_json = open('../config.json')
config_dict = json.load(config_json)
openai.api_key = config_dict["OPENAI_KEY"]

os.environ['OPENAI_API_KEY'] = config_dict["OPENAI_KEY"]

In [10]:
df = pd.read_csv("Data/cleaned_hm.csv")
print("Shape", df.shape)
print("Columns", df.columns)

Shape (100535, 9)
Columns Index(['hmid', 'wid', 'reflection_period', 'original_hm', 'cleaned_hm',
       'modified', 'num_sentence', 'ground_truth_category',
       'predicted_category'],
      dtype='object')


In [11]:
script_lines = df["cleaned_hm"].tolist()[:10000]

In [12]:
# Lectura del archivo .docx y formateo a documento .jsonl
with jsonlines.open('training.jsonl', mode='w') as writer:
    for i, text in enumerate(script_lines):
        if len(text) > 0:
            for line in text.split('.'):
                if len(line) > 0:
                    writer.write({"prompt":"", "completion": ' ' + line })

In [13]:
!openai tools fine_tunes.prepare_data -f training.jsonl -q

Analyzing...

- Your file contains 15807 prompt-completion pairs
- There are 2117 duplicated prompt-completion sets. These are rows: [22, 38, 104, 107, 120, 131, 142, 145, 157, 170, 260, 262, 275, 296, 306, 312, 313, 334, 338, 341, 349, 360, 362, 375, 390, 404, 409, 422, 424, 426, 427, 428, 429, 442, 467, 469, 474, 477, 483, 485, 497, 508, 511, 536, 539, 540, 580, 593, 597, 601, 620, 633, 650, 656, 663, 664, 671, 675, 677, 681, 703, 705, 710, 712, 714, 737, 738, 739, 741, 747, 748, 750, 779, 784, 793, 803, 816, 819, 847, 858, 898, 901, 914, 917, 921, 949, 962, 963, 967, 972, 980, 991, 1010, 1021, 1029, 1045, 1053, 1059, 1065, 1074, 1081, 1089, 1091, 1095, 1102, 1115, 1122, 1135, 1137, 1143, 1145, 1158, 1165, 1170, 1173, 1175, 1177, 1180, 1183, 1187, 1195, 1207, 1229, 1239, 1241, 1251, 1271, 1276, 1280, 1307, 1321, 1343, 1357, 1380, 1384, 1390, 1401, 1403, 1406, 1423, 1440, 1462, 1470, 1475, 1487, 1494, 1498, 1503, 1506, 1509, 1511, 1516, 1539, 1544, 1560, 1564, 1572, 1574, 1578, 1592, 

In [14]:
!openai api fine_tunes.create -t "training_prepared.jsonl" -m davinci --n_epochs=5 --suffix "LabLeniBot"

Uploaded file from training_prepared.jsonl: file-fwNYfRgqL3bKLepTUP24NDyH
Created fine-tune: ft-l5KMBRqm47asBPxCmSZ2QHsJ
Streaming events until fine-tuning is complete...

(Ctrl-C will interrupt the stream, but not cancel the fine-tune)
[2022-08-08 19:35:56] Created fine-tune: ft-l5KMBRqm47asBPxCmSZ2QHsJ
[2022-08-08 19:36:09] Fine-tune failed. Fine-tune will exceed billing hard limit

Job failed. Please contact support@openai.com if you need assistance.



Upload progress:   0%|          | 0.00/12.4M [00:00<?, ?it/s]
Upload progress: 100%|##########| 12.4M/12.4M [00:00<00:00, 12.4Git/s]


In [15]:
!openai api fine_tunes.follow -i  ft-l5KMBRqm47asBPxCmSZ2QHsJ

[2022-08-08 19:35:56] Created fine-tune: ft-l5KMBRqm47asBPxCmSZ2QHsJ
[2022-08-08 19:36:09] Fine-tune failed. Fine-tune will exceed billing hard limit

Job failed. Please contact support@openai.com if you need assistance.
